# Problem 2 - OpenML, Algorithmic Performance Scaling (25 points)

This notebook explores classification tasks using datasets from OpenML, comparing Random Forest and Gradient Boosting classifiers.

---
## Setup: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully.")

---
## Part 1: Load and Summarize Datasets [5 points]

**Task:** Select 2 datasets from OpenML with different number of output classes and summarize their attributes.

In [ ]:
# Dataset 1: Choose a dataset with multi-class classification
# Dataset 2: Choose a dataset with binary classification

print("Loading datasets from OpenML...")
print("="*60)

# TODO: Load Dataset 1
# dataset1 = fetch_openml(...)

# TODO: Load Dataset 2
# dataset2 = fetch_openml(...)

print("Datasets loading section complete.")

In [ ]:
def summarize_dataset(data, target, name):
    """Summarize attributes of a dataset."""
    # TODO: Implement dataset summarization
    # 1. Count features
    # 2. Count instances
    # 3. Count classes
    # 4. Count numerical vs categorical features
    
    summary = {
        'Dataset': name,
        # 'Number of Features': ...,
        # ...
    }
    
    return summary, []

# TODO: Call summarize_dataset for both datasets and display results

---
## Part 2: Training and Evaluation [15 points]

**Task:** 
- Split 80% training / 20% test
- Generate 10 subsets by randomly subsampling 10%, 20%, ..., 100% of training set
- Train Random Forest and Gradient Boosting classifiers
- Measure training time and test accuracy
- Generate learning curves and training time curves

In [ ]:
def prepare_data(data, target):
    """Prepare data for training - handle categorical variables and encode labels."""
    # TODO: Handle encoding of categorical features and target labels
    return data, target

In [ ]:
def run_experiment(X, y, dataset_name, random_state=42):
    """
    Run the training experiment with 10 different training set sizes.
    
    Returns:
        results: dict containing accuracies and training times for both classifiers
    """
    # TODO: Split data into 80% training and 20% test
    
    # TODO: Initialize results dictionary to store metrics
    # results = { ... }
    
    # Training percentages
    percentages = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    
    print(f"\n{'='*70}")
    print(f"Dataset: {dataset_name}")
    print(f"{'='*70}")
    
    for pct in percentages:
        # TODO: Create training subset (handle sampling)
        
        # TODO: Train Random Forest and measure time/accuracy
        
        # TODO: Train Gradient Boosting and measure time/accuracy
        
        # TODO: Store results
        pass
    
    return {}

In [ ]:
def plot_results(results, dataset_name):
    """
    Generate learning curves and training time curves.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # TODO: Extract data from results dictionary
    
    # TODO: Plot Learning Curves (Accuracy vs Data Size) on axes[0]
    
    # TODO: Plot Training Time Curves (Time vs Data Size) on axes[1]
    
    plt.tight_layout()
    plt.show()

In [ ]:
# TODO: Run Experiment and Plot for Dataset 1
# X1, y1 = prepare_data(...)
# results1 = run_experiment(X1, y1, 'Dataset 1')
# plot_results(results1, 'Dataset 1')

In [ ]:
# TODO: Run Experiment and Plot for Dataset 2
# X2, y2 = prepare_data(...)
# results2 = run_experiment(X2, y2, 'Dataset 2')
# plot_results(results2, 'Dataset 2')

---
## Part 3: Analysis and Observations [5 points]

**Task:** Write three main observations about:
1. Scaling of training time
2. Comparison of accuracy between the two classifiers
3. Learning curve behavior

### Your Observations:

**Observation 1: Training Time Scaling**
- [Your answer here]

**Observation 2: Accuracy Comparison**
- [Your answer here]

**Observation 3: Learning Curve Behavior**
- [Your answer here]